# 프로퍼티, 속성과 객체 메소드의 다른 타입들
파이썬 객체의 포르퍼티와 함수는 public이다. 밑줄로 시작하는 속성은 해당 객체에 대해 private을 의미하며, 외부에서 호출하지 않기를 기대하는 것이다

In [13]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60

In [14]:
conn = Connector("postgresql://localhost")

In [15]:
conn.source

'postgresql://localhost'

In [16]:
conn._timeout

60

In [17]:
conn.__dict__

{'source': 'postgresql://localhost', '_timeout': 60}

In [18]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60
    
    def connect(self):
        print("Connecting with {0}s".format(self.__timeout))
        # ...

In [19]:
conn = Connector("postgresql://localhost")

In [20]:
conn.connect()

Connecting with 60s


In [21]:
conn.__timeout

AttributeError: 'Connector' object has no attribute '__timeout'

밑줄 두 개를 사용하면 실제로 파이썬은 다른 이름을 만든다. 이름 맹글링(name mangling)이라 한다.    
이것이 하는 일은 다음과 같은 이름의 속성을 만드는 것이다. "_<class-name>__<attribute_name>" 경우
'_Connector__timeout'이라는 속성이 만들어지며 이러한 속성은 다음과 같이 접근하여 수정할 수 있다.   

In [22]:
vars(conn)

{'source': 'postgresql://localhost', '_Connector__timeout': 60}

In [23]:
conn._Connector__timeout

60

In [24]:
conn._Connector__timeout = 30

In [25]:
conn.connect()

Connecting with 30s


이중 밑줄은 여러 번 확장되는 클래스의 메소드를 이름 충돌 없이 오버라이드 하기 위해 만들어 졌다      
파이썬스러운 코드는 아니다. 밑줄 한번으로 속성을 private으로 정의하는 것이 좋다     

## 프로퍼티
객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용한다. 파이썬에서는 자바 처럼 setter와 getter를 사용하지 않고 프로퍼티를 사용한다.     


In [35]:
import re 

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None                                                            

class User:
    def __init__(self, username):
        self.username = username
        self._email = None 

@property
def email(self):
    return self._email 

@email.setter
def email(self, new_email):
    if not is_valid_email(new_email):
        raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
    self._email = new_email             

@property 메소드는 private 속성인 email 값을 반환한다.  

In [36]:
u1 = User(" jsmith ")
u1.email = "jsmith@"

프로퍼티는 명령-쿼리 분리 원칙( command and query seperatioin )을 따르기 좋은 방법이다.        
명령-분리 원칙 : 무언가의 값을 반환하는 쿼리이거나 둘 중에 하나만 수행해야지 둘 다 둥시에 수행하면 안 된다는 것이다.     
- @property 데코레이터는 무언가에 응답하기 위한 쿼리이고,
- @<property_name>.setter는 무언가를 하기 위한 커맨드이다